# 此代码已放弃，此idea不能有足够合理的理论支撑，也不足以用于减小服务器更新的负担，通常训练与部署是两个不同的服务器

In [1]:
import numpy as np
np.random.seed(42)
import tensorflow.compat.v1 as tf
import keras as K
import pandas as pd
import os

os.environ['CUDA_VISIBLE_DEVICES']='0'
import cv2
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split
from keras.callbacks import ModelCheckpoint
from keras import backend
# from numba import cuda
from keras.models import load_model
import time 
from sklearn.svm import SVC
%matplotlib inline 

Using TensorFlow backend.


In [2]:
# import tensorflow as tf # tf.__version__ == 2.1.0, keras.__version__ == 2.3.1
# https://blog.csdn.net/u012388993/article/details/102573008
# import keras.backend.tensorflow_backend as ktf
# https://blog.csdn.net/zuoyouzouzou/article/details/104329286
import tensorflow.compat.v1 as tf
import tensorflow.compat.v1.keras.backend as ktf

# GPU 显存自动调用
config = tf.ConfigProto()
config.gpu_options.allow_growth=True
session = tf.Session(config=config)
ktf.set_session(session)

In [3]:
train_path = r'flowers_google'
labels_path = "flowers_label.csv"
ids_path = "flowers_idx.csv"

In [4]:
def PrepareAnnotation(ids_path, labels_path, train_path):
    
    ids = pd.read_csv(ids_path)
    labels  = pd.read_csv(labels_path)
    
    annotation_dict = {}

    for i in range(len(ids['id'])):
        image_id = ids['id'][i]
        image_lable = ids['flower_cls'][i]
        annotation_dict[image_id] = image_lable
    
    label_dict = {}
    num_classes = 0

    for i in range(len(labels['flower_class'])):
        label_dict[labels['flower_class'][i]] = labels['label'][i]
        num_classes+=1
        
    for key, value in annotation_dict.items():
        annotation_dict[key] = label_dict[value]
    
    image_pathes = []
    targets = []
    
    for key, value in annotation_dict.items():
        
        image_name = str(key) + ".jpeg"
        image_path = os.path.join(train_path, image_name)
        image_pathes.append(image_path)
        targets.append(value)
    
    return image_pathes, targets, num_classes

In [5]:
image_pathes, targets, num_classes = PrepareAnnotation(ids_path, labels_path, train_path)
val_ratio = 0.2
x_train, x_val, y_train, y_val = train_test_split(image_pathes, targets, test_size=val_ratio, random_state=42, shuffle = True, stratify = targets)
print("num_train = ", len(x_train))
print("num_valid = ", len(x_val))

num_train =  13172
num_valid =  3293


In [6]:
batch_size = 32 # Tesla 上占用9013MB
best_model_root = 'models/models_'+str(batch_size)
model_root = 'models/model_ann'

model_pretrained = load_model(os.path.join(best_model_root, 'Best_Model.h5'))
for layer in model_pretrained.layers:
    if isinstance(layer, K.layers.GlobalAveragePooling2D):
        gap_layer =  layer
model_pretrained = K.models.Model(model_pretrained.inputs, gap_layer.output)

In [7]:
for layer in model_pretrained.layers:
    if isinstance(layer, K.layers.Dense):
        layer.trainable = True
    else:
        layer.trainable = False

# SVM(default_kernel=linear, default C = 1)

### data preparing

In [8]:
alone_id = 100 
x_train_np,y_train_np,x_val_np,y_val_np = map(np.array, [x_train,y_train,x_val, y_val])

func_get_var_103 = lambda x, y: x[y!=alone_id]
x_train_103 = func_get_var_103(x_train_np, y_train_np )
y_train_103 = func_get_var_103(y_train_np, y_train_np )
x_val_103 = func_get_var_103(x_val_np, y_val_np)
y_val_103 = func_get_var_103(y_val_np, y_val_np)
print(x_train_103.shape, y_train_103.shape)

x_train_alone = x_train_np[y_train_np==alone_id]
y_train_alone = y_train_np[y_train_np==alone_id]
x_val_alone = x_val_np[y_val_np==alone_id]
y_val_alone = y_val_np[y_val_np==alone_id]

In [12]:
def files2features(x_train):
    x_svm = []
    from tqdm import tqdm
    batch_num = int(np.ceil(len(x_train)/batch_size))
    bar = tqdm(total = batch_num )
    for i in range(batch_num):
        begin_i = i*batch_size
        end_i = (i+1)*batch_size
        x = x_train[begin_i:end_i]
        x_batch = []

        for i in range(len(x)):
            image = cv2.imread(x[i])
            image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
            image = K.applications.resnet50.preprocess_input(image)
            x_batch.append(image)

        x_batch = np.array(x_batch)
        x_batch = model_pretrained.predict(x_batch).reshape(x_batch.shape[0], -1)
        x_svm.append(x_batch)
        
        bar.update(1)
    x_svm = np.concatenate(x_svm,axis=0)
    return x_svm 

In [13]:
x_train_103_svm = files2features(x_train_103)
y_train_103 = np.array(y_train_103).ravel()

x_val_103_svm = files2features(x_val_103)
y_val_103 = np.array(y_val_103).ravel()

100%|████████████████████████████████████████████████████████████████████████████████| 103/103 [05:43<00:00,  3.33s/it]


## training for 103 flowers

In [ ]:
svm_103 = SVC(kernel ='linear')
svm_103.fit(x_train_103_svm,y_train_103)

## 没有必要了其实，没办法统一，一种花进去，必定会在103种里给出一个结果，虽然可以依靠decision_function来进行修改配合alone的svm

In [ ]:
time_begin = time.time()

x_train_alone_svm = files2features(x_train_alone)
y_train_alone  = np.array(y_train_alone).ravel()

x_val_alone_svm = files2features(x_val_alone)
y_val_alone = np.array(y_val_alone).ravel()

time_end = time.time()

In [ ]:
svm = SVC(kernel = 'linear')

svm.fit(x_svm_103,y_train_103)
time_end = time.time()

print('time used is', (time_end-time_begin)/60)

In [ ]:
x_svm_val = [] 
for i in range(int(np.ceil(len(x_val)/batch_size))):
    begin_i = i*batch_size
    end_i = (i+1)*batch_size
    x = x_val[begin_i:end_i]
    x_batch = []

    for i in range(len(x)):
        image = cv2.imread(x[i])
        image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)
        image = K.applications.resnet50.preprocess_input(image)
        x_batch.append(image)

    x_batch = np.array(x_batch)
    x_batch = model_pretrained.predict(x_batch)
    x_svm_val.append(x_batch)
x_svm_val = np.concatenate(x_svm_val, axis=0)
y_val = np.array(y_val).reshape(-1,1)
test_acc = svm.score(x_svm_val, y_val)

In [ ]:
test_acc

In [ ]:
import joblib 

# test_acc = '0.8757971454600668'
svm_root = 'models/svm/'
if not os.path.exists(svm_root):
    os.makedirs(svm_root)
svm_path = os.path.join(svm_root, '2021-10-02_{}_linear.svm'.format(test_acc))
joblib.dump(svm, svm_path)

In [11]:
svm_08 = SVC(C=0.8, kernel='linear', degree = 1).fit(x_svm,y_train)
test_acc = svm_08.score(x_svm_val, y_val)

d:\other_all\application\python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


In [12]:
test_acc

0.889766170665047

In [13]:
svm_08 = SVC(C=0.9, kernel='linear', degree = 1).fit(x_svm,y_train)
test_acc = svm_08.score(x_svm_val, y_val)
test_acc

d:\other_all\application\python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.889766170665047

In [14]:
svm_08 = SVC(C=1, kernel='linear', degree = 1).fit(x_svm,y_train)
test_acc = svm_08.score(x_svm_val, y_val)
test_acc

d:\other_all\application\python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.889766170665047

In [15]:
svm_08 = SVC(C=1.2, kernel='linear', degree = 1).fit(x_svm,y_train)
test_acc = svm_08.score(x_svm_val, y_val)
test_acc

d:\other_all\application\python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.889766170665047

In [16]:
svm_08 = SVC(C=2, kernel='linear', degree = 1).fit(x_svm,y_train)
test_acc = svm_08.score(x_svm_val, y_val)
test_acc

d:\other_all\application\python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.889766170665047

In [18]:
time_begin = time.time()
svm_08 = SVC(C=1, kernel='linear', degree = 1).fit(x_svm,y_train)
test_acc = svm_08.score(x_svm_val, y_val)
time_end = time.time()
print(test_acc,  (time_end-time_begin)/60)

d:\other_all\application\python\lib\site-packages\sklearn\utils\validation.py:761: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)


0.889766170665047 3.9204971114794414
